In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
from os import path

df = pd.read_csv('../Exploration/demographics.csv')

output_data = '/data/3d/adni-ad-data.pt'
output_labels = '/data/3d/adni-ad-labels.pt'

df = df[df.group == 'AD']
df

,Unnamed: 0,subject,serial_no,age,sex,hospital,group,processed_data_path,type
1617,1617,134940,21,73.0,m,NaN,AD,/data/ADNI/I134940.anat,adni
1618,1618,86261,22,72.0,m,NaN,AD,/data/ADNI/I86261.anat,adni
1619,1619,43071,23,71.0,m,NaN,AD,/data/ADNI/I43071.anat,adni
1632,1632,91210,36,82.0,m,NaN,AD,/data/ADNI/I91210.anat,adni
1633,1633,89979,37,80.0,m,NaN,AD,/data/ADNI/I89979.anat,adni
1634,1634,137267,38,83.0,m,NaN,AD,/data/ADNI/I137267.anat,adni
1656,1656,66206,60,58.0,f,NaN,AD,/data/ADNI/I66206.anat,adni
1657,1657,46608,61,57.0,f,NaN,AD,/data/ADNI/I46608.anat,adni
1658,1658,112391,62,59.0,f,NaN,AD,/data/ADNI/I112391.anat,adni
1687,1687,66778,91,81.0,m,NaN,AD,/data/ADNI/I66778.anat,adni


In [2]:
skullstripped_images_adni = []
files_not_found = []
labels = []
for index, row in df.iterrows():
    if row['type'] == 'fcp':
        print('Processing FCP type dataset')
        if path.exists(row['processed_data_path'] + '/anat/mprage_skullstripped.nii.gz'):
            print(f'Processing {index + 1}')
            print(row['processed_data_path'])
            skullstripped_img = nib.load(row['processed_data_path'] + '/anat/mprage_skullstripped.nii.gz').get_data()
            skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
            print(f'Shape: {skullstripped_img.shape}')
            skullstripped_images_adni.append(skullstripped_img)
            print(row['age'])
            labels.append(row['age'])
        else:
            files_not_found.append(row['processed_data_path'])
    else:
        if path.exists(row['processed_data_path'] + '/T1_biascorr.nii.gz') and path.exists(row['processed_data_path'] + '/T1_fast_seg.nii.gz'):
            print(f'Processing {index + 1}')
            print(row['processed_data_path'])
            orig_img = nib.load(row['processed_data_path'] + '/T1_biascorr.nii.gz').get_data()
            seg_img = nib.load(row['processed_data_path'] + '/T1_fast_seg.nii.gz').get_data()
            skullstripped_img = orig_img * (seg_img > 0)
            skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
            print(f'Shape: {skullstripped_img.shape}')
            skullstripped_images_adni.append(skullstripped_img)
            print(row['age'])
            labels.append(row['age'])
        else:
            files_not_found.append(row['processed_data_path'])

Processing 1618
/data/ADNI/I134940.anat
Shape: (155, 256, 180)
73.0
Processing 1619
/data/ADNI/I86261.anat
Shape: (181, 256, 180)
72.0
Processing 1620
/data/ADNI/I43071.anat
Shape: (155, 256, 180)
71.0
Processing 1633
/data/ADNI/I91210.anat
Shape: (181, 256, 180)
82.0
Processing 1634
/data/ADNI/I89979.anat
Shape: (181, 256, 180)
80.0
Processing 1635
/data/ADNI/I137267.anat
Shape: (154, 256, 180)
83.0
Processing 1657
/data/ADNI/I66206.anat
Shape: (181, 256, 180)
58.0
Processing 1658
/data/ADNI/I46608.anat
Shape: (181, 256, 180)
57.0
Processing 1659
/data/ADNI/I112391.anat
Shape: (181, 256, 180)
59.0
Processing 1688
/data/ADNI/I66778.anat
Shape: (179, 256, 166)
81.0
Processing 1689
/data/ADNI/I119730.anat
Shape: (180, 256, 166)
80.0
Processing 1690
/data/ADNI/I111305.anat
Shape: (179, 256, 166)
82.0
Processing 1691
/data/ADNI/I40356.anat
Shape: (180, 256, 166)
80.0
Processing 1692
/data/ADNI/I66759.anat
Shape: (179, 256, 166)
58.0
Processing 1693
/data/ADNI/I119728.anat
Shape: (180, 256,

Shape: (131, 192, 160)
70.0
Processing 2210
/data/ADNI/I85959.anat
Shape: (137, 192, 160)
72.0
Processing 2211
/data/ADNI/I50487.anat
Shape: (137, 192, 160)
71.0
Processing 2212
/data/ADNI/I123220.anat
Shape: (137, 192, 160)
73.0
Processing 2213
/data/ADNI/I73145.anat
Shape: (131, 192, 160)
81.0
Processing 2214
/data/ADNI/I50468.anat
Shape: (131, 192, 160)
80.0
Processing 2215
/data/ADNI/I123207.anat
Shape: (131, 192, 160)
82.0
Processing 2219
/data/ADNI/I112301.anat
Shape: (131, 192, 160)
79.0
Processing 2220
/data/ADNI/I50426.anat
Shape: (131, 192, 160)
77.0
Processing 2221
/data/ADNI/I72297.anat
Shape: (137, 192, 160)
78.0
Processing 2222
/data/ADNI/I63014.anat
Shape: (136, 192, 160)
88.0
Processing 2223
/data/ADNI/I143815.anat
Shape: (137, 192, 160)
90.0
Processing 2224
/data/ADNI/I109131.anat
Shape: (137, 192, 160)
89.0
Processing 2225
/data/ADNI/I103707.anat
Shape: (137, 192, 160)
77.0
Processing 2226
/data/ADNI/I143799.anat
Shape: (137, 192, 160)
78.0
Processing 2227
/data/ADNI/

Shape: (135, 192, 160)
77.0
Processing 2795
/data/ADNI/I52044.anat
Shape: (135, 192, 160)
77.0
Processing 2796
/data/ADNI/I92169.anat
Shape: (136, 192, 160)
77.0
Processing 2797
/data/ADNI/I118970.anat
Shape: (136, 192, 160)
75.0
Processing 2798
/data/ADNI/I31143.anat
Shape: (136, 192, 160)
73.0
Processing 2799
/data/ADNI/I52031.anat
Shape: (135, 192, 160)
74.0
Processing 2800
/data/ADNI/I92160.anat
Shape: (136, 192, 160)
75.0
Processing 2801
/data/ADNI/I118874.anat
Shape: (136, 192, 160)
73.0
Processing 2802
/data/ADNI/I118877.anat
Shape: (135, 192, 160)
74.0
Processing 2849
/data/ADNI/I89928.anat
Shape: (137, 192, 160)
74.0
Processing 2850
/data/ADNI/I111197.anat
Shape: (136, 192, 160)
74.0
Processing 2851
/data/ADNI/I59543.anat
Shape: (135, 192, 160)
72.0
Processing 2855
/data/ADNI/I59484.anat
Shape: (137, 192, 160)
80.0
Processing 2856
/data/ADNI/I59493.anat
Shape: (136, 192, 160)
81.0
Processing 2857
/data/ADNI/I94405.anat
Shape: (137, 192, 160)
82.0
Processing 2867
/data/ADNI/I91

In [3]:
print('Total data: ', len(df))
print('Available Data:', len(skullstripped_images_adni))
print('Unavailable Data:', len(files_not_found)) 

Total data:  346
Available Data: 346
Unavailable Data: 0


In [4]:
def process_dimension(dimension, image):
    d = image.shape[dimension]
    new_image = None
    if d < 224:
        remaining_dimensions = 224 - d
        if remaining_dimensions % 2 == 0:
            a = int(remaining_dimensions / 2)
            pad_width = ((0,0),(a, a)) if dimension == 1 else ((a,a),(0,0))
            new_image = np.pad(image, pad_width=pad_width, mode="constant", constant_values=0)
        else:
            a = int(remaining_dimensions / 2)
            b = a + 1
            pad_width = ((0,0),(a, b)) if dimension == 1 else ((a,b),(0,0))
            new_image = np.pad(image, pad_width=pad_width, mode="constant", constant_values=0)
    elif d > 224:
        extra_dimensions = d - 224
        if extra_dimensions % 2 == 0:
            a = int(extra_dimensions / 2)
            new_image = image[a:-a,:] if dimension == 0 else image[:, a:-a]
        else:
            a = int(extra_dimensions / 2)
            b = a + 1
            new_image = image[a:-b,:] if dimension == 0 else image[:, a:-b]
    else:
        print('here')
        return image
    return new_image

In [5]:
import torch

new_img_list = []
for i, skullstipped_image_adni in enumerate(skullstripped_images_adni):
    new_img_processed = np.zeros((3,224,224))
    x, y, z = skullstipped_image_adni.shape
    
    new_img = skullstipped_image_adni[int(x/2),:,:]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[0] = new_img
    
    new_img = skullstipped_image_adni[:,int(y/2),:]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[1] = new_img
    
    new_img = skullstipped_image_adni[:,:,int(z/2)]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[2] = new_img
    print(f'Processed {i}, shape: {new_img_processed.shape}')
    new_img_list.append(new_img_processed)

x = np.asarray(new_img_list)
x = torch.Tensor(x).type(torch.float32)
torch.save(x, output_data)

Processed 0, shape: (3, 224, 224)
Processed 1, shape: (3, 224, 224)
Processed 2, shape: (3, 224, 224)
Processed 3, shape: (3, 224, 224)
Processed 4, shape: (3, 224, 224)
Processed 5, shape: (3, 224, 224)
Processed 6, shape: (3, 224, 224)
Processed 7, shape: (3, 224, 224)
Processed 8, shape: (3, 224, 224)
Processed 9, shape: (3, 224, 224)
Processed 10, shape: (3, 224, 224)
Processed 11, shape: (3, 224, 224)
Processed 12, shape: (3, 224, 224)
Processed 13, shape: (3, 224, 224)
Processed 14, shape: (3, 224, 224)
Processed 15, shape: (3, 224, 224)
Processed 16, shape: (3, 224, 224)
Processed 17, shape: (3, 224, 224)
Processed 18, shape: (3, 224, 224)
Processed 19, shape: (3, 224, 224)
Processed 20, shape: (3, 224, 224)
Processed 21, shape: (3, 224, 224)
Processed 22, shape: (3, 224, 224)
Processed 23, shape: (3, 224, 224)
Processed 24, shape: (3, 224, 224)
Processed 25, shape: (3, 224, 224)
Processed 26, shape: (3, 224, 224)
Processed 27, shape: (3, 224, 224)
Processed 28, shape: (3, 224, 

Processed 250, shape: (3, 224, 224)
Processed 251, shape: (3, 224, 224)
Processed 252, shape: (3, 224, 224)
Processed 253, shape: (3, 224, 224)
Processed 254, shape: (3, 224, 224)
Processed 255, shape: (3, 224, 224)
Processed 256, shape: (3, 224, 224)
Processed 257, shape: (3, 224, 224)
Processed 258, shape: (3, 224, 224)
Processed 259, shape: (3, 224, 224)
Processed 260, shape: (3, 224, 224)
Processed 261, shape: (3, 224, 224)
Processed 262, shape: (3, 224, 224)
Processed 263, shape: (3, 224, 224)
Processed 264, shape: (3, 224, 224)
Processed 265, shape: (3, 224, 224)
Processed 266, shape: (3, 224, 224)
Processed 267, shape: (3, 224, 224)
Processed 268, shape: (3, 224, 224)
Processed 269, shape: (3, 224, 224)
Processed 270, shape: (3, 224, 224)
Processed 271, shape: (3, 224, 224)
Processed 272, shape: (3, 224, 224)
Processed 273, shape: (3, 224, 224)
Processed 274, shape: (3, 224, 224)
Processed 275, shape: (3, 224, 224)
Processed 276, shape: (3, 224, 224)
Processed 277, shape: (3, 22

In [6]:
y = np.asarray(labels)
y = torch.Tensor(y).type(torch.float32)
torch.save(y, output_labels)